In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import openmeteo_requests
from openmeteo_sdk import Model, WeatherApiResponse, Variable

import requests_cache
import pandas as pd
from retry_requests import retry

import polars as pl
from datetime import datetime, timedelta, timezone

import logging

logging.basicConfig(level=logging.INFO)

logger = logging.getLogger(__name__)

from pipeline.api_wrapper import Forecast

In [6]:
forecaster = Forecast()
hourly_data, current_data = forecaster.get_data(57.3046898, 25.3627957)
display(hourly_data)
display(current_data)

INFO:pipeline.api_wrapper:Getting data for 57.3046898, 25.3627957


created_at,date,model,latitude,longitude,elevation,temperature,wind_speed,wind_direction,relative_humidity,dew_point,apparent_temperature,precipitation,precipitation_probability,rain,showers,snowfall,snow_depth,weather_code,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high
"datetime[μs, UTC]","datetime[μs, UTC]",str,f64,f64,f64,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
2025-01-10 23:42:45.132660 UTC,2025-01-10 00:00:00 UTC,"""icon_seamless""",57.3125,25.375,115.0,-0.393,2.657066,109.798965,88.0,-2.137431,-4.221079,0.0,3.0,0.0,0.0,0.0,0.01,3.0,98.0,81.0,89.0,55.0
2025-01-10 23:42:45.132660 UTC,2025-01-10 01:00:00 UTC,"""icon_seamless""",57.3125,25.375,115.0,-0.393,2.844293,100.124619,89.0,-1.984247,-4.298726,0.0,28.0,0.0,0.0,0.0,0.01,3.0,99.0,78.0,92.0,45.0
2025-01-10 23:42:45.132660 UTC,2025-01-10 02:00:00 UTC,"""icon_seamless""",57.3125,25.375,115.0,-0.443,3.101613,88.152428,93.0,-1.435988,-4.408221,0.1,80.0,0.0,0.0,0.07,0.01,71.0,100.0,89.0,100.0,72.0
2025-01-10 23:42:45.132660 UTC,2025-01-10 03:00:00 UTC,"""icon_seamless""",57.3125,25.375,115.0,-0.393,3.41321,84.957642,93.0,-1.386396,-4.514822,0.4,100.0,0.0,0.0,0.28,0.01,71.0,100.0,98.0,99.0,100.0
2025-01-10 23:42:45.132660 UTC,2025-01-10 04:00:00 UTC,"""icon_seamless""",57.3125,25.375,115.0,-0.293,3.811824,85.486099,94.0,-1.141201,-4.589255,0.9,100.0,0.0,0.0,0.63,0.01,71.0,100.0,94.0,100.0,100.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2025-01-10 23:42:45.291031 UTC,2025-01-16 19:00:00 UTC,"""dmi_seamless""",57.312023,25.36145,115.0,3.65,4.916299,274.666779,99.0,NaN,-0.450013,0.0,5.0,0.0,0.0,0.0,NaN,1.0,44.0,49.0,0.0,0.0
2025-01-10 23:42:45.291031 UTC,2025-01-16 20:00:00 UTC,"""dmi_seamless""",57.312023,25.36145,115.0,3.2,4.726521,276.072357,98.0,NaN,-0.912511,0.0,7.0,0.0,0.0,0.0,NaN,1.0,38.0,43.0,0.0,0.0
2025-01-10 23:42:45.291031 UTC,2025-01-16 21:00:00 UTC,"""dmi_seamless""",57.312023,25.36145,115.0,2.75,4.428318,276.482971,98.0,NaN,-1.288158,0.0,8.0,0.0,0.0,0.0,NaN,1.0,34.0,38.0,0.0,0.0


created_at,date,model,latitude,longitude,elevation,temperature,wind_speed,wind_direction,relative_humidity,dew_point,apparent_temperature,precipitation,precipitation_probability,rain,showers,snowfall,snow_depth,weather_code,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high
"datetime[μs, UTC]","datetime[μs, UTC]",str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2025-01-10 23:42:45.298520 UTC,2025-01-10 23:30:00 UTC,"""icon_seamless""",57.3125,25.375,115.0,0.45,3.195309,159.863617,86.0,-1.619737,-3.589826,0.0,0.0,0.0,0.0,0.0,0.0,3.0,100.0,84.0,100.0,100.0
2025-01-10 23:42:45.301214 UTC,2025-01-10 23:30:00 UTC,"""gfs_seamless""",57.34462,25.3125,115.0,-0.15,3.712142,175.364624,86.0,-2.209592,-4.541587,0.0,1.0,0.0,0.0,0.0,0.07,3.0,100.0,71.0,100.0,100.0
2025-01-10 23:42:45.303488 UTC,2025-01-10 23:30:00 UTC,"""meteofrance_seamless""",57.299999,25.400002,115.0,0.05,3.573514,162.071991,90.0,-1.394829,-4.157052,0.0,9.0,0.0,0.0,0.0,NaN,3.0,100.0,80.0,82.0,10.0
2025-01-10 23:42:45.305789 UTC,2025-01-10 23:30:00 UTC,"""ecmwf_ifs025""",57.25,25.25,115.0,0.55,3.640055,159.075424,87.0,NaN,-3.687501,0.0,0.0,0.0,0.0,0.0,NaN,3.0,85.0,81.0,31.0,0.0
2025-01-10 23:42:45.308154 UTC,2025-01-10 23:30:00 UTC,"""ecmwf_aifs025""",57.25,25.25,115.0,0.55,3.992493,157.932053,89.0,NaN,-3.828312,0.1,NaN,0.1,0.0,0.0,NaN,51.0,42.0,40.0,10.0,0.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2025-01-10 23:42:45.317091 UTC,2025-01-10 23:30:00 UTC,"""gem_seamless""",57.300003,25.350006,115.0,0.6,3.6,163.0,90.0,-0.851354,-3.542936,0.0,1.0,0.0,0.0,0.0,0.0,2.0,68.0,22.787422,2.079445,0.0
2025-01-10 23:42:45.318689 UTC,2025-01-10 23:30:00 UTC,"""bom_access_global""",57.363281,25.400391,115.0,0.75,3.8,177.0,91.0,-0.55155,-3.453746,0.0,3.0,0.0,0.0,0.0,0.0,3.0,100.0,14.0,75.0,100.0
2025-01-10 23:42:45.320406 UTC,2025-01-10 23:30:00 UTC,"""cma_grapes_global""",57.3125,25.375,115.0,-1.4,3.9,157.380096,96.0,-1.95517,-5.856306,0.0,NaN,0.0,0.0,0.0,0.2,2.0,56.0,52.0,1.0,56.0


In [7]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 57.3046898,
	"longitude": 25.3627957,
	"hourly": ["temperature_2m", "wind_speed_10m", "wind_direction_10m", "relative_humidity_2m"],
    "wind_speed_unit": "ms",
    # "models": all_models
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")



Coordinates 57.30215835571289°N 25.3603515625°E
Elevation 115.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


In [8]:
def model_to_name(code):
    """convert Model to name"""
    for name, value in Model.Model.__dict__.items():
        if value == code and not name.startswith("_"):
            return name
    return None

for response in responses:
    print(model_to_name(response.Model()))

valid_models = list(set([model_to_name(response.Model()) for response in responses]))


best_match


In [9]:
response.Hourly().Variables(0).Variable()

47

In [10]:
import csv

with open("valid_models.csv", 'w') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(valid_models)

In [12]:
dir(response.Hourly().Variables(1))
response.Hourly().Variables(1).Variable()

59

In [13]:
# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
# hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()

# hourly_data = {"date": pd.date_range(
# 	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
# 	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
# 	freq = pd.Timedelta(seconds = hourly.Interval()),
# 	inclusive = "left"
# )}
# hourly_data["temperature_2m"] = hourly_temperature_2m
# hourly_data["wind_speed_10m"] = hourly.Variables(1).ValuesAsNumpy()
# hourly_data["wind_direction_10m"] = hourly.Variables(2).ValuesAsNumpy()
# hourly_data["relative_humidity_2m"] = hourly.Variables(3).ValuesAsNumpy()

# hourly_dataframe = pd.DataFrame(data = hourly_data)
# print(hourly_dataframe)

start = datetime.fromtimestamp(hourly.Time(), timezone.utc)
end = datetime.fromtimestamp(hourly.TimeEnd(), timezone.utc)
freq = timedelta(seconds = hourly.Interval())

hourly_dataframe_pl = pl.select(
    date = pl.datetime_range(start, end, freq, closed = "left"),
    temperature_2m = hourly.Variables(0).ValuesAsNumpy(),
    wind_speed_10m = hourly.Variables(1).ValuesAsNumpy(),
    wind_direction_10m = hourly.Variables(2).ValuesAsNumpy(),
    relative_humidity_2m = hourly.Variables(3).ValuesAsNumpy()
)
hourly_dataframe_pl

date,temperature_2m,wind_speed_10m,wind_direction_10m,relative_humidity_2m
"datetime[μs, UTC]",f32,f32,f32,f32
2025-01-10 00:00:00 UTC,0.0025,2.7,98.0,90.0
2025-01-10 01:00:00 UTC,0.0525,3.3,96.0,90.0
2025-01-10 02:00:00 UTC,0.0025,3.8,91.0,92.0
2025-01-10 03:00:00 UTC,0.0025,3.8,85.0,92.0
2025-01-10 04:00:00 UTC,0.0525,4.0,88.0,96.0
…,…,…,…,…
2025-01-16 19:00:00 UTC,3.65,4.916299,274.666779,99.0
2025-01-16 20:00:00 UTC,3.2,4.726521,276.072357,98.0
2025-01-16 21:00:00 UTC,2.75,4.428318,276.482971,98.0


In [14]:
hourly.Time()

1736467200

In [16]:
import dagster
class MDConfig(dagster.Config):
    # hourly_data_path: str = os.path.join(os.path.dirname(__file__), "data", "hourly_data.csv")
    hourly_data_columns: list[str] = ["temperature_2m", "wind_speed_10m", "wind_direction_10m", "relative_humidity_2m"]

config = MDConfig()

params = {
	"latitude": 57.3046898,
	"longitude": 25.3627957,
	"hourly": config.hourly_data_columns,
    # "hourly": ["temperature_2m", "wind_speed_10m", "wind_direction_10m", "relative_humidity_2m"],
    "wind_speed_unit": "ms",
    "timezone": "Europe/Riga"
}

for key, value in params.items():
    print(f"{key}: {value}")
    print(type(value))

responses = openmeteo.weather_api(url, params=params)


latitude: 57.3046898
<class 'float'>
longitude: 25.3627957
<class 'float'>
hourly: ['temperature_2m', 'wind_speed_10m', 'wind_direction_10m', 'relative_humidity_2m']
<class 'list'>
wind_speed_unit: ms
<class 'str'>
timezone: Europe/Riga
<class 'str'>


In [17]:
extremes = {
    "north": 58.085137,
    "south": 55.674971,
    "east": 28.241043,
    "west": 20.970154
}

In [18]:
(extremes["north"] - extremes["south"]) / (extremes["east"] - extremes["west"])

0.33148161112073143

In [19]:
n_long = 45

In [20]:
import geopandas as gpd

gpd.tools.geocode("Riga, Latvia")

,geometry,address
0,POINT (24.10518 56.9494),"Rīga, Latvija"


In [21]:
import reverse_geocode
reverse_geocode.get((57.3046898, 25.3627957))

{'country_code': 'LV',
 'city': 'Priekuļi',
 'latitude': 57.315,
 'longitude': 25.36147,
 'population': 2532,
 'state': 'Cēsis Municipality',
 'county': 'Priekuļu pagasts',
 'country': 'Latvia'}

In [22]:
import shapely
gpd.tools.reverse_geocode([shapely.geometry.Point(57.3046898, 25.3627957),shapely.geometry.Point(57.3046898, 25.3627957)])

,geometry,address
0,POINT EMPTY,None
1,POINT EMPTY,None


In [24]:
gpd.tools.reverse_geocode(  
    [shapely.geometry.Point(-71.0594869, 42.3584697), shapely.geometry.Point(-77.0365305, 38.8977332),
     shapely.geometry.Point(57.315, 25.36147), shapely.geometry.Point(57.3046898, 25.3627957)
    ]
)

,geometry,address
0,POINT (-71.05958 42.35862),"18-32, Tremont Street, 02108, Tremont Street, ..."
1,POINT (-77.03655 38.8977),"White House, 1600, Pennsylvania Avenue Northwe..."
2,POINT EMPTY,None
3,POINT EMPTY,None


In [26]:
gpd.read_file("shapfiles/lv_10km.shp")


,geometry
0,"POLYGON ((4870000 3670000, 4870000 3680000, 48..."
1,"POLYGON ((4870000 3680000, 4870000 3690000, 48..."
2,"POLYGON ((4870000 3690000, 4870000 3700000, 48..."
3,"POLYGON ((4870000 3700000, 4870000 3710000, 48..."
4,"POLYGON ((4870000 3710000, 4870000 3720000, 48..."
...,...
1295,"POLYGON ((5440000 3850000, 5440000 3860000, 54..."
1296,"POLYGON ((5450000 3800000, 5450000 3810000, 54..."
1297,"POLYGON ((5450000 3810000, 5450000 3820000, 54..."
1298,"POLYGON ((5450000 3820000, 5450000 3830000, 54..."


In [27]:
countries = gpd.read_file("shapfiles/world-administrative-boundaries.shp")


In [28]:
countries.head()

,iso3,status,color_code,name,continent,region,iso_3166_1_,french_shor,geometry
0,MNP,US Territory,USA,Northern Mariana Islands,Oceania,Micronesia,MP,Northern Mariana Islands,"MULTIPOLYGON (((145.63331 14.91236, 145.62412 ..."
1,None,Sovereignty unsettled,RUS,Kuril Islands,Asia,Eastern Asia,None,Kuril Islands,"MULTIPOLYGON (((146.68274 43.70777, 146.66664 ..."
2,FRA,Member State,FRA,France,Europe,Western Europe,FR,France,"MULTIPOLYGON (((9.4475 42.68305, 9.45014 42.63..."
3,SRB,Member State,SRB,Serbia,Europe,Southern Europe,RS,Serbie,"POLYGON ((20.26102 46.11485, 20.31403 46.06986..."
4,URY,Member State,URY,Uruguay,Americas,South America,UY,Uruguay,"POLYGON ((-53.3743 -33.74067, -53.39917 -33.75..."


In [29]:
countries[countries["iso3"] == "LVA"]

,iso3,status,color_code,name,continent,region,iso_3166_1_,french_shor,geometry
86,LVA,Member State,LVA,Latvia,Europe,Northern Europe,LV,Lettonie,"POLYGON ((27.37206 57.53564, 27.425 57.53194, ..."


In [30]:
import numpy as np

n_long = 45
n_lat = 15

longitudes = np.linspace(extremes["west"], extremes["east"], n_long)
latitudes = np.linspace(extremes["south"], extremes["north"], n_lat)

longitudes
latitudes

points = [shapely.geometry.Point(long, lat) for long in longitudes for lat in latitudes]

lv_grid = gpd.GeoDataFrame(geometry=points)
lv_grid.crs = "EPSG:4326"

lv_grid


,geometry
0,POINT (20.97015 55.67497)
1,POINT (20.97015 55.84713)
2,POINT (20.97015 56.01928)
3,POINT (20.97015 56.19144)
4,POINT (20.97015 56.36359)
...,...
670,POINT (28.24104 57.39652)
671,POINT (28.24104 57.56867)
672,POINT (28.24104 57.74083)
673,POINT (28.24104 57.91298)


In [31]:
contains = lv_grid.geometry.within(countries[countries["iso3"] == "LVA"].geometry.iloc[0])

lv_grid = lv_grid[contains].reset_index(drop=True)


In [32]:
lv_grid.geometry.iloc[0].coords[0][0]


20.970154

In [33]:
lv_grid["rev_geocode"] = lv_grid.geometry.apply(lambda x: reverse_geocode.get((x.coords[0][1], x.coords[0][0])))

In [34]:
lv_grid["latitude"] = lv_grid.rev_geocode.apply(lambda x: x["latitude"])
lv_grid["longitude"] = lv_grid.rev_geocode.apply(lambda x: x["longitude"])

lv_grid["full_name"] = lv_grid.rev_geocode.apply(lambda x: f"{x.get('state', '')}{', ' if x.get('state') else ''}{x.get('county', '')}{', ' if x.get('county') else ''}{x.get('city', '')}")

lv_grid = lv_grid[lv_grid.rev_geocode.apply(lambda x: x.get("country_code") == "LV")].reset_index(drop=True)


In [35]:
lv_grid.drop_duplicates(subset=["full_name"])

,geometry,rev_geocode,latitude,longitude,full_name
0,POINT (20.97015 56.36359),"{'country_code': 'LV', 'city': 'Nīca', 'latitu...",56.34601,21.06401,"South Kurzeme Municipality, Nīcas pagasts, Nīca"
1,POINT (21.1354 56.19144),"{'country_code': 'LV', 'city': 'Rucava', 'lati...",56.16314,21.16156,"South Kurzeme Municipality, Rucavas pagasts, R..."
3,POINT (21.1354 56.53574),"{'country_code': 'LV', 'city': 'Grobiņa', 'lat...",56.53521,21.16782,"South Kurzeme Municipality, Grobiņa"
6,POINT (21.30065 56.53574),"{'country_code': 'LV', 'city': 'Lieģi', 'latit...",56.58173,21.33399,"South Kurzeme Municipality, Tadaiķu pagasts, L..."
8,POINT (21.30065 56.88005),"{'country_code': 'LV', 'city': 'Pāvilosta', 'l...",56.88791,21.18593,"South Kurzeme Municipality, Pāvilosta"
...,...,...,...,...,...
286,POINT (27.41481 56.01928),"{'country_code': 'LV', 'city': 'Dagda', 'latit...",56.09512,27.53723,"Krāslava Municipality, Dagda"
291,POINT (27.41481 56.88005),"{'country_code': 'LV', 'city': 'Kārsava', 'lat...",56.78405,27.68829,"Ludza Municipality, Kārsava"
297,POINT (27.58005 56.36359),"{'country_code': 'LV', 'city': 'Ludza', 'latit...",56.53958,27.71891,"Ludza Municipality, Ludza"
301,POINT (27.58005 57.05221),"{'country_code': 'LV', 'city': 'Viļaka', 'lati...",57.18458,27.67220,"Balvi Municipality, Viļaka"


In [36]:
lv_grid[lv_grid["full_name"].str.startswith("Cēsis")].full_name.tolist()

['Cēsis Municipality, Stalbes pagasts, Stalbe',
 'Cēsis Municipality, Līgatne',
 'Cēsis Municipality, Līgatne',
 'Cēsis Municipality, Stalbes pagasts, Stalbe',
 'Cēsis Municipality, Drabešu pagasts, Drabeši',
 'Cēsis Municipality, Drabešu pagasts, Drabeši',
 'Cēsis Municipality, Cēsis',
 'Cēsis Municipality, Drabešu pagasts, Drabeši',
 'Cēsis Municipality, Priekuļu pagasts, Priekuļi',
 'Cēsis Municipality, Vecpiebalgas pagasts, Vecpiebalga',
 'Cēsis Municipality, Vecpiebalgas pagasts, Vecpiebalga',
 'Cēsis Municipality, Vecpiebalgas pagasts, Vecpiebalga',
 'Cēsis Municipality, Vecpiebalgas pagasts, Vecpiebalga']

In [38]:
lv_grid.drop_duplicates(subset=["full_name"])[["full_name", "latitude", "longitude"]].to_csv("lv_grid.csv", index=False)